# [Issue 98](https://github.com/mjuenema/python-terrascript/issues/98): Change to .update() in 0.8.0? 

## Problem

> In our Terrascript 0.6.1 scripts, we have a bunch of functions that define pieces of the configuration, and we 
> assemble them like this:

```python
ts = terrascript.Terrascript()
ts += terrascript.terraform(
  backend = terrascript.backend(
    "s3",
    **config['backend']['s3']
    )
  )
ts.update(tsumami.provider.define(config))
ts.update(tsumami.s3.define(config))
ts.update(tsumami.security_groups.define(config))
```

> where `config` is a dict based on a collection of YAML config files.
>
> Each of those functions looks like this:

```python
def define(config):
  my_ts = terrascript.Terrascript()
  if 's3-bucket' in config:
    for (resource, data) in config['s3-bucket'].items():
      # code that does various things
      my_ts += terrascript.resource.aws.aws_s3_bucket(
        resource,
        bucket = bucketname,
        **data
        )
  return(my_ts)
```

> That used to work in 0.6.1, but doesn't in 0.8.0; it seems that each `ts.update(something)` replaces the
> previous one, and only the last one ends up in the config when we write out `str(ts)` at the end.
>
> What's the right way to do this in 0.8.0?

## Analysis

The problem is caused by having removed the ``Terrascript.update()`` method in release 0.8.0.

```console
$ git diff 0.6.1 0.8.0 terrascript/__init__.py
-    def update(self, terrascript2):
-        if isinstance(terrascript2, Terrascript):
-            for item in terrascript2._item_list:
-                self.__add__(item)
-        else:
-            raise TypeError('{0} is not a Terrascript instance.'.format(
-                type(terrascript2)))
```

In fact, there is still a ``Terrascript.update()`` method but it is now inherited from the Python dictionary class which works of course completely differently.


**Terrascript 0.8.0**
```python
class Terrascript(dict):
    ...
```

**Terrascript 0.6.1**
```python
class Terrascript(object)
```

## Fix

The obvious fix is to restore the ``Terrascript.update()`` method. This is not a simply copy and paste 
job as the ``Terrascript._item_list`` attribute has also been removed.

In Terrascript 0.6.1 ``Terrascript.__item_list`` was managed behind the scenes. Terrascript 0.8.0 removed all this logic with the objective of making it much simpler.

**Terrascript 0.6.1**

```python
class Terrascript(object):
    """Top-level container for Terraform configurations."""

    def __init__(self):
        ...
        self._item_list = []
        
    def __add__(self, item):
        ...
        if not isinstance(item, Terrascript):
            if item in self._item_list:
                self._item_list.remove(item)
            self._item_list.append(item)
```

Since release 0.8.0 the ``Terrascript`` class is just a Python dictionary with extra smarts the best approach would be to make ``Terrascript.update()`` perform a "nested dictionary merge". That should hopefully provide the
desired result.